### Stappenplan
Data splitten op verschillende tijdsranges en dan de volgende features uitrekenen:
- Number of events in each sub process
- Number of events with each document type
- Average, minimum and maximum amount of time spent in a sub process 
- Average minimum and maximum amount of time spent in a document type
- For each sub process: amount of time spent in the sub process
- For each document type: the time spent in such document type
- Amount of (un)successful events until a point
- Total time passed
- Month started

- Amount applied changes
- 

In [1]:
import pandas as pd
import numpy as np
import utils
import pyarrow
from datetime import datetime, timedelta

READ_PARQUET = True
pd.set_option('display.max_columns', None)

In [2]:
import importlib
utils = importlib.reload(utils)  # reloads the utils module without restarting kernel

In [3]:
if READ_PARQUET:
    print("Reading parquet.")
    log = utils.load_parquet()
else:
    print("Reading csv and savings as parquet.")
    df = utils.load_csv()
    utils.save_parquet(df)

Reading parquet.


In [4]:
from copy import deepcopy
# drop uncomplete cases (2017 mostly) and thus do not have a complete label

#cases_df = deepcopy(df.loc[df['Complete Timestamp'].dt.year < 2017])
df = deepcopy(log)

In [5]:
cases = pd.DataFrame(df['Case ID'].unique())
cases = cases.rename(columns = {0: 'Case ID'})

In [6]:
cases

,Case ID
0,8b99873a6136cfa6
1,d038e6a5fcaed7a4
2,d0c4a4241daa1d89
3,beb37ac32c4dfd60
4,accaf25e4ea7629a
...,...
43804,eade921440d732f4
43805,74e72c66adc05ae4
43806,75e5c760d8399329
43807,50d70b51ed3efcdc


In [7]:
df.head()

,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) amount_applied0,(case) amount_applied1,(case) amount_applied2,(case) amount_applied3,(case) applicant,(case) application,(case) area,(case) basic payment,(case) cross_compliance,(case) department,(case) greening,(case) number_parcels,(case) payment_actual0,(case) payment_actual1,(case) payment_actual2,(case) payment_actual3,(case) penalty_ABP,(case) penalty_AGP,(case) penalty_AJLP,(case) penalty_AUVP,(case) penalty_AVBP,(case) penalty_AVGP,(case) penalty_AVJLP,(case) penalty_AVUVP,(case) penalty_B16,(case) penalty_B2,(case) penalty_B3,(case) penalty_B4,(case) penalty_B5,(case) penalty_B5F,(case) penalty_B6,(case) penalty_BGK,(case) penalty_BGKV,(case) penalty_BGP,(case) penalty_C16,(case) penalty_C4,(case) penalty_C9,(case) penalty_CC,(case) penalty_GP1,(case) penalty_JLP1,(case) penalty_JLP2,(case) penalty_JLP3,(case) penalty_JLP5,(case) penalty_JLP6,(case) penalty_JLP7,(case) penalty_V5,(case) penalty_amount0,(case) penalty_amount1,(case) penalty_amount2,(case) penalty_amount3,(case) program-id,(case) redistribution,(case) rejected,(case) risk_factor,(case) selected_manually,(case) selected_random,(case) selected_risk,(case) small farmer,(case) year,(case) young farmer,activity,concept:name,docid,doctype,eventid,lifecycle:transition,note,subprocess,success
0,8b99873a6136cfa6,Payment application-Application-mail income,0;n/a,2015/05/08 00:00:00.000,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,False,mail income,mail income,-18008611495569447,Payment application,NaN,complete,none,Application,True
1,8b99873a6136cfa6,Payment application-Application-mail valid,0;n/a,2015/05/08 00:00:00.000,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,False,mail valid,mail valid,-18008611495569447,Payment application,NaN,complete,none,Application,True
2,8b99873a6136cfa6,Entitlement application-Main-mail valid,0;n/a,2015/05/08 00:00:00.000,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,False,mail valid,mail valid,-18008615298673397,Entitlement application,NaN,complete,none,Main,True
3,8b99873a6136cfa6,Entitlement application-Main-mail valid,0;n/a,2015/05/08 00:00:00.000,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,False,mail valid,mail valid,-18008615298673397,Entitlement application,NaN,complete,none,Main,True
4,8b99873a6136cfa6,Parcel document-Main-initialize,fb5fa8,2015/06/10 11:16:28.000,Variant 1832,1832,960.35,NaN,NaN,NaN,b3b1bafcf8a5c359,8b99873a6136cfa6,2.6994,True,0.0,e7,True,3,960.35,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,NaN,NaN,NaN,215,True,False,1.0,False,False,False,True,2015,Fa

### For overview's sake : only contain event attributes for now (remove when making final dataframe (I think))

In [8]:
cols = [c for c in df.columns if c.lower()[:6] != '(case)']

df=df[cols]

### Adding  Sub Process

In [9]:
def column_sub_process(data):
    data['Sub_Process'] = data['Activity'].apply(lambda x: x.split('-', 2)[:2])
    data['Sub_Process'] = [', '.join(map(str, l)) for l in data['Sub_Process']]
    return data

In [10]:
df = column_sub_process(df)

### Adding start date to both dataframes

In [11]:
start_date = df.groupby(by = ['Case ID'])['Complete Timestamp'].agg(['min']).rename(columns={'min':'StartDate'})

In [12]:
start_date = start_date.reset_index()
start_date

,Case ID,StartDate
0,0002505cb62792e4,2015/04/28 00:00:00.000
1,0002a55a6130cec8,2015/04/17 00:00:00.000
2,0004ff62053a60ce,2015/05/11 00:00:00.000
3,000612b48d30de74,2015/05/06 00:00:00.000
4,0006cc909ce508b0,2016/04/28 00:00:00.000
...,...,...
43804,fffa80507910b904,2017/05/10 00:00:00.000
43805,fffa93db29ee0841,2016/05/12 00:00:00.000
43806,fffab10d958a3e4d,2016/05/11 00:00:00.000
43807,fffabee101da7ecb,2017/05/11 00:00:00.000


In [13]:
cases = pd.merge(cases, start_date, on = 'Case ID')
cases['StartDate'] = pd.to_datetime(cases['StartDate']).dt.date

In [14]:
df = pd.merge(df, start_date, on = 'Case ID')
df['Complete Timestamp'] = pd.to_datetime(df['Complete Timestamp']).dt.date
df['StartDate'] = pd.to_datetime(df['StartDate']).dt.date

In [15]:
df.head()

,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,activity,concept:name,docid,doctype,eventid,lifecycle:transition,note,subprocess,success,Sub_Process,StartDate
0,8b99873a6136cfa6,Payment application-Application-mail income,0;n/a,2015-05-08,Variant 1832,1832,mail income,mail income,-18008611495569447,Payment application,NaN,complete,none,Application,True,"Payment application, Application",2015-05-08
1,8b99873a6136cfa6,Payment application-Application-mail valid,0;n/a,2015-05-08,Variant 1832,1832,mail valid,mail valid,-18008611495569447,Payment application,NaN,complete,none,Application,True,"Payment application, Application",2015-05-08
2,8b99873a6136cfa6,Entitlement application-Main-mail valid,0;n/a,2015-05-08,Variant 1832,1832,mail valid,mail valid,-18008615298673397,Entitlement application,NaN,complete,none,Main,True,"Entitlement application, Main",2015-05-08
3,8b99873a6136cfa6,Entitlement application-Main-mail valid,0;n/a,2015-05-08,Variant 1832,1832,mail valid,mail valid,-18008615298673397,Entitlement application,NaN,complete,none,Main,True,"Entitlement application, Main",2015-05-08
4,8b99873a6136cfa6,Parcel document-Main-initialize,fb5fa8,2015-06-10,Variant 1832,1832,initialize,initialize,-72051858488795160,Parcel document,-7.205186e+16,complete,none,Main,True,"Parcel document, Main",2015-05-08


### Calculate events per sub-process

Dit misschien veranderen naar dat hij alleen de sub-processen count die niet main of application zijn, correleert heel erg met doctype. 

In [ ]:
df.head()

In [ ]:
def events_per_sub_process(data, days):
    subprocesses= list(df['Sub_Process'].unique())
    final_dict = dict()
    for index in range(len(data)):
        subdict = dict((el, 0) for el in subprocesses)
        case = data['Case ID'][index]
        subset = df[df['Case ID'] == case]
        subset = subset[subset['Complete Timestamp'] <= (subset['StartDate'] + timedelta(days))]
        counts = subset['Sub_Process'].value_counts().to_dict()
        for sub in subdict: 
            if sub in counts: 
                subdict[sub] = counts[sub] 
            final_dict[case] = subdict
    return final_dict
    #filter such that all activities occur within specified amount of days
    #count amount of occurences for this sub_process
    #put it in a nicely and automatically generated column

In [ ]:
dictionary = events_per_sub_process(cases[0:5], 33)

In [ ]:
dictionary

In [ ]:
def merge_dict_with_dataframe(dictionary, data):
    dictdata = pd.DataFrame.from_dict(dictionary, orient = 'index')
    dictdata.reset_index(inplace = True)
    dictdata = dictdata.rename(columns = {'index': 'Case ID'})
    dataframe = pd.merge(data, dictdata, on = 'Case ID')
    return dataframe

In [ ]:
cases = merge_dict_with_dataframe(dictionary, cases)

In [ ]:
df.columns

### Calculate events per doctype

In [113]:
def events_per_doctype(data, days):
    doctypes= list(df['doctype'].unique())
    final_dict = dict()
    for index in range(len(data)):
        subdict = dict((el, 0) for el in doctypes)
        case = data['Case ID'][index]
        subset = df[df['Case ID'] == case]
        subset = subset[subset['Complete Timestamp'] <= (subset['StartDate'] + timedelta(days))]
        counts = subset['doctype'].value_counts().to_dict()
        for sub in subdict: 
            if sub in counts: 
                subdict[sub] = counts[sub] 
            final_dict[case] = subdict
    return final_dict

In [114]:
dictionary = events_per_doctype(cases[0:3], 20)

In [115]:
dictionary

{'8b99873a6136cfa6': {'Payment application': 2,
  'Entitlement application': 2,
  'Parcel document': 0,
  'Control summary': 0,
  'Reference alignment': 0,
  'Department control parcels': 0,
  'Inspection': 0,
  'Geo parcel document': 0},
 'd038e6a5fcaed7a4': {'Payment application': 2,
  'Entitlement application': 2,
  'Parcel document': 0,
  'Control summary': 0,
  'Reference alignment': 0,
  'Department control parcels': 0,
  'Inspection': 0,
  'Geo parcel document': 0},
 'd0c4a4241daa1d89': {'Payment application': 2,
  'Entitlement application': 2,
  'Parcel document': 0,
  'Control summary': 0,
  'Reference alignment': 0,
  'Department control parcels': 0,
  'Inspection': 0,
  'Geo parcel document': 0}}

In [ ]:
cases = merge_dict_with_dataframe(dictionary, cases)

### Amount of time spent in the sub process and average, min, max

In [157]:
def time_in_sub_process(data, days):
    subprocesses= list(df['Sub_Process'].unique())
    final_dict = dict()
    for index in range(len(data)):
        subdict = dict((el, 0) for el in subprocesses)
        subdict = {f'Days in {k}': v for k, v in subdict.items()}
        case = data['Case ID'][index]
        subset = df[df['Case ID'] == case]
        subset = subset[subset['Complete Timestamp'] <= (subset['StartDate'] + timedelta(days))]
        times = subset.groupby(['Case ID','Sub_Process'])['Complete Timestamp'].agg(['min','max']).rename(columns={'min':'first','max':'last'})
        times['timediff'] = times['last'] - times['first']
        times.reset_index(inplace = True)
        times.timediff = times['timediff'].dt.days
        timesdict = dict(zip(times.Sub_Process, times.timediff))
        timesdict = {f'Days in {k}': v for k, v in timesdict.items()}
        for sub in subdict: 
            if sub in timesdict: 
                subdict[sub] = timesdict[sub] 
#                 subdict = {f'Days in {k}': v for k, v in subdict.items()}
            final_dict[case] = subdict
       # final_dict = {}
    return final_dict

In [171]:
dictionary = time_in_sub_process(cases[0:10], 180)
dictionary

{'8b99873a6136cfa6': {'Days in Payment application, Application': 0,
  'Days in Entitlement application, Main': 176,
  'Days in Parcel document, Main': 40,
  'Days in Control summary, Main': 0,
  'Days in Reference alignment, Main': 0,
  'Days in Department control parcels, Main': 0,
  'Days in Entitlement application, Change': 0,
  'Days in Payment application, Main': 0,
  'Days in Payment application, Change': 0,
  'Days in Inspection, Remote': 0,
  'Days in Inspection, On': 0,
  'Days in Entitlement application, Objection': 0,
  'Days in Payment application, Objection': 0,
  'Days in Geo parcel document, Main': 0,
  'Days in Geo parcel document, Declared': 0,
  'Days in Geo parcel document, Reported': 0},
 'd038e6a5fcaed7a4': {'Days in Payment application, Application': 4,
  'Days in Entitlement application, Main': 171,
  'Days in Parcel document, Main': 122,
  'Days in Control summary, Main': 0,
  'Days in Reference alignment, Main': 0,
  'Days in Department control parcels, Main':

### Amount of time spent in each document type and average, min, max

In [179]:
def time_in_doctype(data, days):
    doctypes= list(df['doctype'].unique())
    final_dict = dict()
    for index in range(len(data)):
        docdict = dict((el, 0) for el in doctypes)
        docdict = {f'Days in {k}': v for k, v in docdict.items()}
        case = data['Case ID'][index]
        subset = df[df['Case ID'] == case]
        subset = subset[subset['Complete Timestamp'] <= (subset['StartDate'] + timedelta(days))]
        times = subset.groupby(['Case ID','doctype'])['Complete Timestamp'].agg(['min','max']).rename(columns={'min':'first','max':'last'})
        times['timediff'] = times['last'] - times['first']
        times.reset_index(inplace = True)
        times.timediff = times['timediff'].dt.days
        timesdict = dict(zip(times.doctype, times.timediff))
        timesdict = {f'Days in {k}': v for k, v in timesdict.items()}
        for doc in docdict: 
            if doc in timesdict: 
                docdict[doc] = timesdict[doc] 
            final_dict[case] = docdict
    return final_dict

In [180]:
time_in_doctype(cases[0:1], 180)

{'8b99873a6136cfa6': {'Days in Payment application': 0,
  'Days in Entitlement application': 176,
  'Days in Parcel document': 40,
  'Days in Control summary': 0,
  'Days in Reference alignment': 0,
  'Days in Department control parcels': 0,
  'Days in Inspection': 0,
  'Days in Geo parcel document': 0}}

In [121]:
test_dict = {f'Days in {k}': v for k, v in test_dict.items()}

In [122]:
test_dict

{'Days in Control summary, Main': 0,
 'Days in Entitlement application, Main': 0,
 'Days in Parcel document, Main': 5,
 'Days in Payment application, Application': 0}

In [64]:
test2['timediff']

0    0.0
1    0.0
2    5.0
3    0.0
Name: timediff, dtype: float64

In [56]:
test2

,index,Case ID,Sub_Process,first,last,timediff
0,0,8b99873a6136cfa6,"Control summary, Main",2015-06-10,2015-06-10,0 days 00:00:00.000000000
1,1,8b99873a6136cfa6,"Entitlement application, Main",2015-05-08,2015-05-08,0 days 00:00:00.000000000
2,2,8b99873a6136cfa6,"Parcel document, Main",2015-06-10,2015-06-15,5 days 00:00:00.000000000
3,3,8b99873a6136cfa6,"Payment application, Application",2015-05-08,2015-05-08,0 days 00:00:00.000000000


In [45]:
#area_dict = dict(zip(lakes.area, lakes.count))
test_dict = dict(zip(test2.Sub_Process, test2.timediff))
data['result'] = data['result'].map(lambda x: x.lstrip('+-').rstrip('aAbBcC'))

In [49]:
test2.reset_index(inplace = True)
#dictdata.reset_index(inplace = True)
test2['timediff']

0   0 days
1   0 days
2   5 days
3   0 days
Name: timediff, dtype: timedelta64[ns]

In [48]:
test_dict

{'Control summary, Main': Timedelta('0 days 00:00:00'),
 'Entitlement application, Main': Timedelta('0 days 00:00:00'),
 'Parcel document, Main': Timedelta('5 days 00:00:00'),
 'Payment application, Application': Timedelta('0 days 00:00:00')}

In [47]:
test_dict['Control summary, Main']

Timedelta('0 days 00:00:00')

### Amount of total and (un)succesful events

# Check if columns are numerical not categorical or string